In [1]:
import pandas as pd
import chardet

In [2]:
#carga manual de dataset
dataset1 = 'Datasets_relevamiento_precios/precios_semana_20200413.csv' #Csv precio_semana
dataset2 = 'Datasets_relevamiento_precios/precios_semana_20200503.json' #json precio_semana
dataset3 = 'Datasets_relevamiento_precios/precios_semana_20200518.txt'#txt precio_semana
#dataset4 = 'Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx' #xls precio_semana
dataset5 = 'Datasets_relevamiento_precios/producto.parquet' #parquet producto
dataset6 = 'Datasets_relevamiento_precios/sucursal.csv' #csv sucursal



In [3]:
#abro el archivo y veo el encoding - 
#TAREA: buscar si hay otra forma autom. de hacerlo
def detect_encoding(dataset):
    '''
    Esta función sirve para detectar los tipo de enconding para distintos formatos
    '''
    with open(dataset, 'rb') as f:
        enc = chardet.detect(f.read())
    return enc['encoding']         

        ** Transformación dataset 1  **

In [5]:
#dataset precio semana 1
precio_semana20200413 = pd.read_csv(dataset1, delimiter=",", encoding=detect_encoding(dataset1))

In [6]:
#ver la estrucura del df
precio_semana20200413

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248
...,...,...,...
472161,139.99,9569753142128,25-1-1
472162,34.99,9795403001143,25-1-1
472163,312.50,9990385651922,5-1-3
472164,312.50,9990385651939,5-1-3


In [7]:
def busca_nulos(dataframe):
    '''
    Muestra todos los valores nulos que se encuentren en un dataframe
    '''
    return dataframe.isnull().sum()


In [8]:
#se buscan valores nulos en dataframe 1
busca_nulos(precio_semana20200413)

precio         13
producto_id    15
sucursal_id    15
dtype: int64

In [9]:
# el % de nulos es relativamente bajo. Se mantienen para un futuro posible tratamiento en la étapa de análisis 
# y teniendo en cuenta el objetivo del negocio

In [134]:
#el precio es de tipo flotante por lo que no es necesaria ninguna modificacion
#precio_semana202005030200413['precio']=precio_semana202005030200413['precio'].astype(float)

In [213]:
#no parece haber nada raro en esto
precio_semana20200413['precio'].describe()

count    472153.000000
mean        183.644026
std         353.948880
min           0.120000
25%          65.900000
50%         117.000000
75%         210.000000
max       34417.150000
Name: precio, dtype: float64

In [136]:
#transformación de dataframe a csv
#precio_semana202005030200413.to_csv('producto_mod.csv', index=False)

        ** Transformación dataset 2 **

In [10]:
#dataset precio semana 2
precio_semana20200503 = pd.read_json(dataset2)

In [11]:
precio_semana20200503

,precio,producto_id,sucursal_id
0,29.9,0000000002288,2-1-187
1,39.9,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59
...,...,...,...
397729,139.99,9569753142128,25-1-1
397730,34.99,9795403001143,25-1-1
397731,312.5,9990385651922,5-1-5
397732,312.5,9990385651939,5-1-5


In [176]:
#se buscan valores nulos en dataframe 2
busca_nulos(precio_semana20200503)

precio         0
producto_id    0
sucursal_id    0
dtype: int64

In [12]:
#tranformar a numero cuando la serie tiene distintos tipos de datos
precio_semana20200503['precio']=pd.to_numeric(precio_semana20200503.precio)

In [13]:
#pasar a flotante el precio
precio_semana20200503['precio'].astype(float)

0          29.90
1          39.90
2         499.99
3         539.99
4         519.99
           ...  
397729    139.99
397730     34.99
397731    312.50
397732    312.50
397733    198.90
Name: precio, Length: 397734, dtype: float64

    ** Transformación dataset ** 

In [14]:
#dataset precio semana 3
#pd.read_fwf(dataset4)
precio_semana20200518 = pd.read_csv(dataset3, delimiter="|", encoding=detect_encoding(dataset3))


In [15]:
precio_semana20200518

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430
...,...,...,...
415288,149.99,9569753142128,25-1-1
415289,34.99,9795403001143,25-1-1
415290,312.50,9990385651922,5-1-4
415291,312.50,9990385651939,5-1-4


In [181]:
#se buscan valores nulos en dataframe 3
busca_nulos(precio_semana20200518)

precio         1960
producto_id       6
sucursal_id       6
dtype: int64

In [182]:
#guardo los valores nulos. Ya que no hay casi información es imposible dar con estos datos. 
precio_semana20200518[precio_semana20200518['producto_id'].isna()]

,precio,producto_id,sucursal_id
375118,203.0,NaN,NaN
375157,NaN,NaN,NaN
375193,240.0,NaN,NaN
375278,NaN,NaN,NaN
375357,NaN,NaN,NaN
375397,NaN,NaN,NaN


In [16]:
#se elimina estos datos ya que no aportan información ni se considera necesario su posterior tratamiento
precio_semana20200518 = precio_semana20200518.dropna(subset=['producto_id'])

In [184]:
#la cantidad de nulos en precio no es tan signidicativa. Igualmente se podría hacer un tratamiento si se lo desea una vez ingestadas la tablas
(1960/415293)


0.004719559443573574

In [17]:
#pasar a flotante el precio
precio_semana20200518['precio'].astype(float)

0          29.90
1          32.90
2          36.90
3          39.90
4         499.99
           ...  
415288    149.99
415289     34.99
415290    312.50
415291    312.50
415292    198.90
Name: precio, Length: 415287, dtype: float64

        ** Transformación Dataset 4 y 5 **

In [18]:
#dataset precio semana 4
precios_xlsx = pd.read_excel('Datasets_relevamiento_precios/precios_semanas_20200419_20200426.xlsx', sheet_name=None)

In [19]:
# unir todas las hojas en un solo dataframe y limpiarlo desde allí
precio_semana20200419_20200426 = pd.concat(precios_xlsx, ignore_index=True)

In [20]:
#se detectaron algunos valores atípicos que por falta de tiempo no se les da el traramiento
precio_semana20200419_20200426['precio'].describe()

count    9.339090e+05
mean     2.024225e+12
std      2.746591e+14
min      2.200000e-01
25%      1.200000e+02
50%      4.690000e+02
75%      1.794000e+03
max      4.399890e+16
Name: precio, dtype: float64

In [188]:
precio_semana20200419_20200426 

,precio,sucursal_id,producto_id
0,399.00,2-1-092,2288.0
1,299.00,2-1-206,2288.0
2,399.00,2-2-241,2288.0
3,49999.00,9-1-430,205870.0
4,53999.00,9-2-4,205870.0
...,...,...,...
937447,139.99,2001-01-25 00:00:00,9569753142128
937448,34.99,2001-01-25 00:00:00,9795403001143
937449,312.50,2003-01-05 00:00:00,9990385651922
937450,312.50,2003-01-05 00:00:00,9990385651939


In [21]:
precio_semana20200419_20200426 = precio_semana20200419_20200426[['precio', 'producto_id', 'sucursal_id']]

In [22]:
precio_semana20200419_20200426

,precio,producto_id,sucursal_id
0,399.00,2288.0,2-1-092
1,299.00,2288.0,2-1-206
2,399.00,2288.0,2-2-241
3,49999.00,205870.0,9-1-430
4,53999.00,205870.0,9-2-4
...,...,...,...
937447,139.99,9569753142128,2001-01-25 00:00:00
937448,34.99,9795403001143,2001-01-25 00:00:00
937449,312.50,9990385651922,2003-01-05 00:00:00
937450,312.50,9990385651939,2003-01-05 00:00:00


In [191]:
#se buscan valores nulos en dataframe 4
busca_nulos(precio_semana20200419_20200426)

precio          3543
producto_id    13519
sucursal_id        0
dtype: int64

In [356]:
#nulos_productos_id 
13519/937452

0.0144210050221238

In [ ]:
#En un caso real se hablaría con el cliente y se pediría datos de la misma sucursal ya sea una semana anterior y posterior 
#se propone cruzar datos con esa tabla e imputar los datos faltantes a mismos productos, con misma sucursal, pero de un periódo cercano.
#caso de no contar con esa info, se pueden utilizar datos de otra sucursal que maneje precios similares (mismas caracteristicas, ej: hipermercado, mercado etc) y región

In [23]:
# se elimina el cero que figura en el id_producto y no debería. Además se agregan los números necesarios para el código EAN
precio_semana20200419_20200426['producto_id'] = precio_semana20200419_20200426['producto_id'].apply(lambda x: str(x).replace('.0',''))
precio_semana20200419_20200426['producto_id'] = precio_semana20200419_20200426['producto_id'].str.zfill(13)

In [24]:
precio_semana20200419_20200426

,precio,producto_id,sucursal_id
0,399.00,0000000002288,2-1-092
1,299.00,0000000002288,2-1-206
2,399.00,0000000002288,2-2-241
3,49999.00,0000000205870,9-1-430
4,53999.00,0000000205870,9-2-4
...,...,...,...
937447,139.99,9569753142128,2001-01-25 00:00:00
937448,34.99,9795403001143,2001-01-25 00:00:00
937449,312.50,9990385651922,2003-01-05 00:00:00
937450,312.50,9990385651939,2003-01-05 00:00:00


In [25]:
#convierto el dato str para poder manipularlo mejor
precio_semana20200419_20200426['sucursal_id']=precio_semana20200419_20200426['sucursal_id'].astype(str)

In [27]:
def norm_fecha (x):
    '''
    Esta funcion intenta normalizar los datos que vienen cargados como datetime en alguna columna
    toma cada argumento seguido de '-' y los retorna los valores en el órden correcto
    '''
    
    if ' 00:00:00' in x:
        x = x.replace(' 00:00:00','')
        val_1 = x.split('-')[0]
        val_2 = x.split('-')[1]
        val_3 = x.split('-')[2]
       
        x = val_3 + '-' + val_2 + '-' + val_1
    return x

In [28]:
#se estandarizan datos 
precio_semana20200419_20200426['sucursal_id']=precio_semana20200419_20200426['sucursal_id'].apply(norm_fecha)

In [29]:
#pasar a flotante el precio
precio_semana20200419_20200426['precio'].astype(float)

0           399.00
1           299.00
2           399.00
3         49999.00
4         53999.00
            ...   
937447      139.99
937448       34.99
937449      312.50
937450      312.50
937451      198.90
Name: precio, Length: 937452, dtype: float64

In [30]:
precio_semana20200419_20200426

,precio,producto_id,sucursal_id
0,399.00,0000000002288,2-1-092
1,299.00,0000000002288,2-1-206
2,399.00,0000000002288,2-2-241
3,49999.00,0000000205870,9-1-430
4,53999.00,0000000205870,9-2-4
...,...,...,...
937447,139.99,9569753142128,25-01-2001
937448,34.99,9795403001143,25-01-2001
937449,312.50,9990385651922,05-01-2003
937450,312.50,9990385651939,05-01-2003


In [31]:
#pasar a flotante el precio
precio_semana20200419_20200426['precio'].astype(float)

0           399.00
1           299.00
2           399.00
3         49999.00
4         53999.00
            ...   
937447      139.99
937448       34.99
937449      312.50
937450      312.50
937451      198.90
Name: precio, Length: 937452, dtype: float64

In [32]:
#dataset producto
producto = pd.read_parquet(dataset5, engine='pyarrow')

In [117]:
#tamaño de la tabla producto.shape (72038, 7)
#la tabla no tiene productos duplicados --> producto.drop_duplicates(inplace=True)
#shape despues de borrar duplicados es la misma #tamaño de la tabla (72038, 7)

In [33]:
#tamaño de la tabla
producto.shape

(72038, 7)

In [34]:
#ver valores nulos 
producto.isnull().sum() #mismo resultado si uso .isna 

id                  0
marca               2
nombre              2
presentacion        2
categoria1      72034
categoria2      72034
categoria3      72034
dtype: int64

In [ ]:
#las columnas últimas 3 columnas tienen un % muy bajo de valores y es muy difícil inferir cuales serían los datos faltantes sólo en base a la información que tenemos.
#Elimino las columnas y si fuera un caso real, le muestro los siguientes porcentajes al cliente para su consideración. 
# Además le preguntaría si esa info es relevante y de serlo, propondría armar una tabla de categorías (id producto y categoría)

In [35]:
#guardo el total en una variable
total = producto.shape[0]


In [175]:
producto.categoria1.isnull().sum()

72034

In [176]:
nulos_cat1 = round((72034*100/total), 2)
nulos_cat2 = round((72034*100/total), 2)
nulos_cat3 = round((72034*100/total), 2)
print("total de faltantes categoría 1:", nulos_cat1, '%')
print("total de faltantes categoría 2:", nulos_cat3, '%')
print("total de faltantes categoría 3:", nulos_cat2, '%')

total de faltantes categoría 1: 99.99 %
total de faltantes categoría 2: 99.99 %
total de faltantes categoría 3: 99.99 %


In [36]:
#dropeo las ultimas columnas porque aportan poca y nada información. Se puede plantear más adelante generar una tabla de categoría
producto.drop(['categoria1', 'categoria2', 'categoria3'], axis=1, inplace=True)

In [37]:
#veo cuales son los porductos sin 'nombre'
producto[producto['nombre'].isnull()]
#coinciden con los 2 null de marca

,id,marca,nombre,presentacion
53619,7798024299045,None,None,None
55798,7798061190213,None,None,None


In [38]:
#seteo el valor de datos faltantes en "sin dato"
producto['nombre'].fillna("Sin Dato", inplace = True)
producto['marca'].fillna("Sin Dato", inplace = True)
producto['presentacion'].fillna("Sin Dato", inplace = True)

In [39]:
#elimino los números y unidades del nombre ya que parece un error de carga 
producto.nombre = producto.nombre.astype(str).apply(lambda x: x.strip('0123456789UnKgGr '))

In [78]:
producto

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima,1.0 un
1,0000000002288,LA ANÓNIMA,Perejil Atado La Anonima,1.0 un
2,0000000205870,SIN MARCA,Ojo de Bife,1.0 kg
3,0000000205894,SIN MARCA,Milanesa de Peceto Novillito,1.0 kg
4,0000000205955,SIN MARCA,Chiquizuela Novillito,1.0 kg
...,...,...,...,...
72033,9569753142128,DELI-SITAS,Milhojas Cobertura de Chocolate Blanco Deli-Sitas,500.0 gr
72034,9795403001143,MAYO,Mini Pizzetas Mayo,12.0 un
72035,9990385651922,DANA,Te Negro en Hebras Lata Dana,50.0 gr
72036,9990385651939,DANA,Te Verde en Hebras Lata Dana,50.0 gr


        ** Transformación de la tabla sucursal **

In [40]:
#dataset sucursal
sucursal = pd.read_csv(dataset6, delimiter=",", encoding=detect_encoding(dataset6))

In [44]:
sucursal.head()

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,lat,lng,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado


In [110]:
# no hay valores nulos
sucursal.isnull().sum()

id                     0
comercioId             0
banderaId              0
banderaDescripcion     0
comercioRazonSocial    0
provincia              0
localidad              0
direccion              0
lat                    0
lng                    0
sucursalNombre         0
sucursalTipo           0
dtype: int64

In [13]:
#la tabla de Provincia contiene un Código que a simple vista parece coincidir con cada Provincia. 
#no se agrega info

In [45]:
#cambio nombre de las columnas lat y lng para que sea más claro
sucursal.rename(columns={'lat': 'Latitud', 
                            'lng': 'Longitud'}, inplace=True)


In [112]:
sucursal

,id,comercioId,banderaId,banderaDescripcion,comercioRazonSocial,provincia,localidad,direccion,Latitud,Longitud,sucursalNombre,sucursalTipo
0,1-1-7,1,1,Super MAMI,Dinosaurio S.A.,AR-X,SALSIPUEDES,E53 1011 None,-31.126667,-64.295250,Super Mami 4,Hipermercado
1,10-1-1,10,1,Hipermercado Carrefour,INC S.A.,AR-B,San Isidro,Bernardo De Irigoyen 2647,-34.491345,-58.589025,San Isidro,Hipermercado
2,10-1-10,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Hurlingham,Av. Vergara 1910,-34.620610,-58.633769,Villa Tesei,Hipermercado
3,10-1-11,10,1,Hipermercado Carrefour,INC S.A.,AR-B,Malvinas Argentinas,Av. Arturo Illia 3770,-34.528883,-58.701631,Malvinas Argentinas,Hipermercado
4,10-1-112,10,1,Hipermercado Carrefour,INC S.A.,AR-A,Salta,20 De Febrero 37,-24.789072,-65.413699,Salta,Hipermercado
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,9-3-5277,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-U,COMODORO RIVADAVIA,Yrigoyen Hipolito 0,-45.873300,-67.493500,Jumbo-Comodoro,Hipermercado
2329,9-3-5626,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,GENERAL PACHECO,Boulogne Sur Mer 1275,-34.474500,-58.625700,Jumbo Pacheco Novo,Hipermercado
2330,9-3-5961,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-C,CIUDAD AUTONOMA BUENOS AIRES,Avenida Santa Fe 4950,-34.577200,-58.430000,Jumbo Av. Santa Fé,Supermercado
2331,9-3-628,9,3,Jumbo,Jumbo Retail Argentina S.A.,AR-B,SAN FERNANDO,Avenida Del Libertador Gral San Martin 2271,-34.446900,-58.545700,Jumbo San Fernando,Supermercado


        ** CONEXIÓN A LA BASE DE DATOS CREADA EN MYSQL **

In [ ]:
#se recomienda en el futuro crear modelos para hacer el mapeo correspondiente. 
#Por falta de tiempo no se llego a modularizar, pero se lo recomienda para mejorar el proyecto

In [48]:
#from pandas.io import sql
import sqlalchemy as db
import pymysql

In [128]:
#Se prepara la conexión para la prueba. Luego hay que pasar las variables de entorno a un .env e importarlas en este .ipynb
database_username='root' 
database_password='Martini12!'
database_ip='localhost'
database_name='supermercados'
database_conection=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
conect=database_conection.connect()
metadata=db.MetaData()

        ** CARGA de dataframes tranformados **

In [ ]:
#Por una cuestión de tiempos, se crearon las tablas directamente en el gestor de base de datos de MySQL. 
#Idealmente se deberían crear módelos (con clase base de sqlAlquemy) para mapear las tablas.

In [87]:
''' def carga_bd(dataframe, tabla):
    try:
        dataframe.to_sql(con=conect, name=tabla, if_exists='append', index=False)
        print("La carga fue exitosa!")
    except:
        print("Ups! no se ha podido cargar los datos")
    finally:
        conect.close() 
'''

In [111]:
# carga de la tabla producto (original = /producto.parquet)
producto.to_sql(con=conect, name='producto', if_exists='append', index=False)

72038

In [112]:
#carga del la tabla sucursal (original /sucursal.csv)
sucursal.to_sql(con=conect, name='sucursal', if_exists='append', index=False)

2333

In [125]:
# 1) carga del la tabla venta. Semana_20200413 (original /precios_semana_20200413.csvv)
precio_semana20200413.to_sql(con=conect, name='venta', if_exists='append', index=False)

472166

In [126]:
# 2y3) carga del la tabla venta. Semana_20200419_20200426 (original /precios_semanas_20200419_20200426.xlsx)
precio_semana20200419_20200426.to_sql(con=conect, name='venta', if_exists='append', index=False)

937452

In [127]:
# 4) carga del la tabla venta. Semana 20200503 (original /precios_semana_20200503.json)
precio_semana20200503.to_sql(con=conect, name='venta', if_exists='append', index=False)

397734

In [129]:
# 5) carga del la tabla venta. Semana 2020018 (original /precios_semana_20200518.txt)
precio_semana20200518.to_sql(con=conect, name='venta', if_exists='append', index=False)

415287

        ** cierre de la conexión **

In [131]:
conect.close()